In [ ]:
import mercury as mr # for widgets
from matplotlib import pyplot as plt # for chart
from qablet_contracts.rate.swaption import swaption_timetable, bermuda_swaption_timetable
from qablet_contracts.bond.zero import zcb_timetable, zbp_timetable
import numpy as np
from qablet.hullwhite.fd import HWFDModel
from qablet.base.fixed import FixedModel

In [ ]:
# Create Mercury App
app = mr.App(title="Qablet: Rate Instruments", description="Samples app in Mercury", allow_download=False)

mercury.App

In [ ]:
# Select Instrument type and create timetable
inst_type = mr.Select(label="Type of Instrument", value="Zero Bond",
                      choices=["Vanilla Swaption", "Bermuda Swaption", "Zero Bond", "Zero Bond Put"])

_ = mr.Note(text="**Instrument Terms**")

if inst_type.value == "Vanilla Swaption":
    fixed_rate = mr.Slider(label="Fixed Rate (%)", value=3, min=0, max=10).value / 100
    start = mr.Slider(label="Start in (days)", value=30, min=0, max=180).value / 365
    maturity = mr.Select(label="Maturity (Years)", value="2", choices=["1", "2", "5", "10"]).value
    freq = mr.Select(label="Payments per year", value="4", choices=["1", "2", "4"]).value

    num_periods = int(maturity) * int(freq)
    periods = start + np.linspace(0, int(maturity), num = num_periods + 1)
    timetable = swaption_timetable("USD", periods, strike_rate=fixed_rate)

elif inst_type.value == "Bermuda Swaption":
    fixed_rate = mr.Slider(label="Fixed Rate (%)", value=3, min=0, max=10).value / 100
    start = mr.Slider(label="Start in (days)", value=30, min=0, max=180).value / 365
    maturity = mr.Select(label="Maturity (Years)", value="2", choices=["1", "2", "5", "10"]).value
    freq = mr.Select(label="Payments per year", value="4", choices=["1", "2", "4"]).value

    num_periods = int(maturity) * int(freq)
    periods = start + np.linspace(0, int(maturity), num = num_periods + 1)
    timetable = bermuda_swaption_timetable("USD", periods, strike_rate=fixed_rate)

elif inst_type.value == "Zero Bond":
    maturity = mr.Select(label="Maturity (Years)", value="2", choices=["1", "2", "5", "10"]).value
    timetable = zcb_timetable("USD", float(maturity))

elif inst_type.value == "Zero Bond Put":
    opt_maturity = mr.Select(label="Option Maturity (Years)", value="2", choices=["1", "2", "5", "10"]).value
    bond_maturity = mr.Select(label="Bond Maturity (Years)", value="2", choices=["1", "2", "5", "10"]).value
    strike = mr.Slider(label="Strike (%)", value=90, min=10, max=200).value / 100
    timetable = zbp_timetable("USD", float(opt_maturity), float(bond_maturity), strike)

mercury.Select

**Instrument Terms**

mercury.Select

## TimeTable

In [ ]:

print(timetable["events"].to_pandas()) # Display Timetable

  track  time op  quantity unit
0         2.0  +       1.0  USD


In [ ]:
# Edit Discount rates (flat).
rate = mr.Slider(label="Discount Rate (%)", value=3, min=0, max=10).value / 100

times = np.array([0.0, 11])
rates = np.array([rate, rate])  # i.e. 4%, 4%, 4.5%, 5%
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))


mercury.Slider

In [ ]:
# Select Model and model parameters
_ = mr.Note(text="**Model**")

model_type = mr.Select(label="Model", value="Hull-White FD",
                      choices=["Hull-White FD", "Fixed"])

if model_type.value == "Hull-White FD":
    model = HWFDModel()

    # Make the model inputs, at least Mean Rev and Vol, editable.
    dataset = {
        "BASE": "USD",
        "ASSETS": {"USD": discount_data},
        "FD": {
            "TIMESTEP": 1 / 500,
            "MAX_X": 0.10,
            "N_X": 1000,
        },
        "HW": {
            "MEANREV": 0.1,
            "VOL": 0.03,
        },
    }
elif model_type.value == "Fixed":
    dataset = {
        "BASE": "USD",
        "ASSETS": {"USD": discount_data},
    }
    model = FixedModel()

**Model**

mercury.Select

In [ ]:
# Calculate Price
price, _ = model.price(timetable, dataset)
mr.NumberBox(data=round(price, 4), title="Price")


**Price**